# プライベート属性よりパブリックな属性が好ましい

In [1]:
class MyObject:
  def __init__(self):
    self.public_field = 5
    self.__private_field = 10

  def get_private_field(self):
    return self.__private_field

In [2]:
foo = MyObject()
assert foo.public_field == 5

In [5]:
assert foo.get_private_field() == 10

In [ ]:
# エラーになる
foo.__private_field


AttributeError: 'MyObject' object has no attribute '__private_field'

In [7]:
class MyOtherObject:
  def __init__(self):
    self.__private_field = 71

  @classmethod
  def get_private_field_of_instance(cls, instance):
    return instance.__private_field
  
bar = MyOtherObject()
assert MyOtherObject.get_private_field_of_instance(bar) == 71

In [8]:
# サブクラスは、スーパークラスのプライベートフィールドにアクセスできない
class MyParentObject:
  def __init__(self):
    self.__private_field = 71

class MyChildObject(MyParentObject):
  def get_private_field(self):
    return self.__private_field

In [ ]:
baz = MyChildObject()
baz.get_private_field()

AttributeError: 'MyChildObject' object has no attribute '_MyChildObject__private_field'

プライベート属性の振る舞いは、属性名の単純な変換で実装される。
Python コンパイラが MyChildObject.get_private_field メソッドのようなプライベート属性アクセスを、__private_field の代わりに _MyChildObject__private_field にアクセスするように変換される。

この例では、__private_field が MyParentObject.__init__ のみで定義されていて、プライベート属性の本当の名前が _MyParentObject__private_field であることを意味する。
サブクラスから親のプライベート属性をアクセスすると、変換された属性名が合致しないという単純な理由で失敗する。

この方式がわかっていれば、サブクラスや外部から許可をもらわなくても、任意のクラスのプライベート属性に簡単にアクセスができる。

In [10]:
assert baz._MyParentObject__private_field == 71

オブジェクトの属性辞書を見れば、プライベート属性が実は返還後に表示される名前で登録されている

In [11]:
print(baz.__dict__)

{'_MyParentObject__private_field': 71}


なぜ、プライベート属性の構文は、厳密な可視性を強制しないのか。

最も単純な回答は、Python のモットー「みんな大人なんだから」である。

Python プログラマは、オープンであることの便益がクローズであることへの誘惑を上回ると信じている

In [14]:
class MyStringClass:
  def __init__(self, value):
    self.__value = value

  def get_value(self):
    return str(self.__value)

foo = MyStringClass(5)
assert foo.get_value() == '5'

プライベート属性を選んだことで、サブクラスでの置き換えや拡張性を面倒で脆弱にしている。

サブクラスは、絶対に必要となれば、プライベートフィールドにアクセスすることも可能

In [16]:
class MyIntegerSubClass(MyStringClass):
  def get_value(self):
    return int(self._MyStringClass__value)

foo = MyIntegerSubClass(5)
assert foo.get_value() == 5

クラス階層が下の方で変わったらどうなるか

In [ ]:
class MyBaseClass:
  def __init__(self, value):
    self.__value = value
  
  def get_value(self):
    return self.__value
  
class MyStringClass(MyBaseClass):
  def get_value(self):
    return str(super().get_value()) # 更新済み
  
class MyIntegerSubClass(MyStringClass):
  def get_value(self):
    # __value は、MyStringClass に定義されていない
    # MyBaseClass に定義されている
    # つまり、self.MyStringClass__value という属性は存在しない
    # アクセスしたいのは、self._MyBaseClass__value（これは動作する）
    return int(self.MyStringClass__value) # 更新していない

__value属性は、スーパークラス MyBaseClass で代入されていて、スーパークラス MyStringClass においてではない。
したがって、プライベート変数への参照 self._MyStringClass__value は、MyIntegerSubclass では動かない

In [ ]:
foo = MyIntegerSubClass(5)
foo.get_value()

AttributeError: 'MyIntegerSubClass' object has no attribute 'MyStringClass__value'

一般に、Protected 属性を用いてサブクラスに多くのことをやらせて失敗する方がまだましである。

保護フィールドにそれぞれに文書化して、サブクラスでのどの内部 API を使うことができ、どの API を使うべきでないかを説明しておくこと

これは、他のプログラマに対してだけでなく、将来自分がコードを安全に拡張する場合にも指針となる

In [24]:
class MyStringClass:
  def __init__(self, value):
    # This stores the user-supplied value for the object.
    # It should be coercible to a string. Once assigned for
    # the object it should be treated as immutable.
    self._value = value

プライベート属性を使うことを真面目に考えるべきなのは、サブクラスとの名前の衝突を心配しないといけないときだけ

In [26]:
class ApiClass:
  def __init__(self):
    self._value = 5

  def get(self):
    return self._value
  
class Child(ApiClass):
  def __init__(self):
    super().__init__()
    self._value = 'hello' # 衝突

a = Child()
print(f'{a.get()} and {a._value} should be different')

hello and hello should be different


このような衝突をなくすために、スーパークラスでプライベート属性を用いて、サブクラスと重複する属性名がないようにする

In [27]:
class ApiClass:
  def __init__(self):
    self.__value = 5

  def get(self):
    return self.__value
  
class Child(ApiClass):
  def __init__(self):
    super().__init__()
    self._value = 'hello' # 衝突

a = Child()
print(f'{a.get()} and {a._value} are different')

5 and hello are different
